In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
import re
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
previous_messages=[]
patterns = [{'intent' : 'age', 'pattern' : r"(i|I) am (\d+).*"}, {'intent' : 'age', 'pattern' : r"(m|M)y age is (\d+).*"}, {'intent' : 'name', 'pattern' : r"(i|I) am ([A-Z][a-z ]*)"}, {'intent' : 'name', 'pattern' : r"(m|M)y name is ([A-Z][a-z ]*)"}, {'intent' : 'living place', 'pattern' : r"(i|I) (live|stay) (in|at) (.*)"}, {'intent' : 'fav place', 'pattern' : r"(m|M)y favou*rite place is (.*)"}, {'intent' : 'fav color', 'pattern' : r"(m|M)y favou*rite color is (.*)"}, {'intent' : 'hobbies', 'pattern' : r"(m|M)y hobb(y|ies) (is|are|:| :) (.*)"}]
info = {}
info = {}

In [2]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
            for i,w in enumerate(words):
                if w == s: 
                    # assign 1 if current word is in the vocabulary position
                    bag[i] = 1
                    if show_details:
                        print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    previous_messages.append(sentence)
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.75
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [3]:
def getResponse(ints, intents_json):
    print(ints)
    if len(ints)==0:
        return random.choice(intents_json['intents'][3]['responses'])
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            extra = ""
            if tag == 'what is my age':
                if 'age' in info:
                    extra = f"\nYou told me previously that your age is {info['age']}"
                else:
                    extra = "\nSorry I don't know... Can you tell me your age ?"
            elif tag == 'what is my name':
                if 'name' in info:
                    extra = f"\nYou told me previously that your name is {info['name']}"
                else:
                    extra = "\nSorry I don't know... Can you tell me your name ?"
            elif tag == 'greeting':
                if not re.match(r"[HIAhia](i|ello|ey|nyone|s).*", previous_messages[-1]):
                    return "Please give Valid Input"
            elif tag == 'what is my fav place':
                if 'favorite place' in info:
                    extra = f"\nYou told me previously that your favorite place is {info['favorite place']}"
                else:
                    extra = "\nSorry I don't know... Can you tell me your favorite place ?"
            elif tag == 'what is my fav color':
                if 'favorite color' in info:
                    extra = f"\nYou told me previously that your favorite color is {info['favorite color']}"
                else:
                    extra = "\nSorry I don't know... Can you tell me your favorite color ?"    
            result = random.choice(i['responses']) + extra
            break
    return result
def chatbot_response(text):
    if re.match(r"who.*re you.*", text.lower()):
        return "I am GLaDOS, a simple conversational ChatBot. What's your good name ?"
    if re.match(r"where.*you.*(live|stay|reside).*", text.lower()):
        return "I am just a simple AI program, created in a Laptop... I just reside in a system stack. Where do you live ?"
    match = re.match(r"my ([a-zA-Z ]+) is (.*)", text.lower())
    if match:
        info[match.group(1)] = match.group(2)
        return "Nice your "+match.group(1)+" is "+match.group(2)
    if re.match("what[ ']*s your fav[ourite]* place", text.lower()):
        if 'favorite place' in info:
            return f"I don't really have one... Yours in {info['favorite place']} is cool !"
        return "I don't really have one... what's yours ? Tell me like this \"My favorite place is ...\""
    if re.match("what[ ']*s your fav[ourite]* color", text.lower()):
        if 'favorite color' in info:
            return f"Mine is Black ... Yours is {info['favorite color']} which is cool !"
        return "Mine is Black ... what's yours ? Tell me like this \"My favorite color is ...\""
    if re.match("(what.*|tell me) your hobb.*", text.lower()):
        if 'hobbies' in info:
            return f"I'm just a program, I don't have any ... Your hobbies: {info['hobbies']} are cool !"
        return "I'm just a program, I don't have any ... what're yours ? Tell me like this \"My hobbies are ...\""
    if re.match("(what.*|tell me) my hobb.*", text.lower()):
        if 'hobbies' in info:
            return f"I'm just a program, I don't have any ... Your hobbies: {info['hobbies']} are cool !"
        return "I'm just a program, I don't have any ... what're yours ? Tell me like this \"My hobbies are ...\""
    if re.match("what.* my fav.*colou*rs*", text.lower()):
        if 'favorite color' in info:
            return f"My favorite color is Black ... Yours: {info['favorite color']} is also nice !"
        return "My favorite color is Black ... what're yours ? Tell me like this \"My favorite color is ...\""
    for i in patterns:
        match = re.match(i['pattern'], text)
        if match:
            if i['intent']=='age':
                age = match.group(2)
                info['age'] = age
                return f"Wow your age is {age}"
            if i['intent']=='name':
                name = match.group(2)
                info['name'] = name
                return f"Nice to meet you {name}! Tell me how can I be of help ?"
            if i['intent']=='living place':
                living_place = match.group(4)
                info['living place'] = living_place
                return f"Oh you live in {living_place}! Tell me how can I be of help ?"
            if i['intent']=='fav place':
                fav_place = match.group(2)
                info['favorite place'] = fav_place
                return f"Ooo your favorite place is {fav_place}! Tell me how can I be of help ?"
            if i['intent']=='fav color':
                fav_color = match.group(2)
                info['favorite color'] = fav_place
                return f"Ooo your favorite color is {fav_color}! Mine is Black.. Tell me how can I be of help ?"
            if i['intent']=='hobbies':
                hobbies = match.group(4)
                info['hobbies'] = hobbies
                return f"Ooo your hobbies are {hobbies}! I just don't have any... Tell me how can I be of help ?"
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [4]:
#Creating GUI with tkinter
import tkinter as tk
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5, bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff', command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="black",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[{'intent': 'greeting', 'probability': '0.9477177'}]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[{'intent': 'age', 'probability': '0.998804'}]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
[{'intent': 'what is your name', 'probability': '0.855966'}]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
[{'intent': 'how_are_you', 'probability': '0.99998295'}]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
[{'intent': 'I am fine', 'probability': '0.9999813'}]


: 